<a href="https://colab.research.google.com/github/AdamLenning/AdamLenning.github.io/blob/master/Optuna_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 6.7 MB/s 
     |████████████████████████████████| 81 kB 5.1 MB/s 
     |████████████████████████████████| 210 kB 17.1 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 150 kB 14.2 MB/s 
     |████████████████████████████████| 113 kB 16.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=5f796df7103d431b51506bddbafa0d16161f4638fb12c47cbdabd56ccc7bfaea
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.
In this example, we optimize the validation accuracy of hand-written digit recognition using
PyTorch and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.
"""

import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-03-26 07:24:41,903] A new study created in memory with name: no-name-810b6be7-fb0b-446e-90b8-80b4ca2e48e3
[I 2022-03-26 07:24:47,762] Trial 0 finished with value: 0.69140625 and parameters: {'n_layers': 3, 'n_units_l0': 92, 'dropout_l0': 0.4335383403077778, 'n_units_l1': 107, 'dropout_l1': 0.230662770740349, 'n_units_l2': 31, 'dropout_l2': 0.3626633180680596, 'optimizer': 'Adam', 'lr': 0.02177704062583298}. Best is trial 0 with value: 0.69140625.
[I 2022-03-26 07:24:53,568] Trial 1 finished with value: 0.7140625 and parameters: {'n_layers': 1, 'n_units_l0': 38, 'dropout_l0': 0.2502375777626525, 'optimizer': 'Adam', 'lr': 0.00015123517974967082}. Best is trial 1 with value: 0.7140625.
[I 2022-03-26 07:24:59,280] Trial 2 finished with value: 0.82109375 and parameters: {'n_layers': 1, 'n_units_l0': 44, 'dropout_l0': 0.26545310274881984, 'optimizer': 'Adam', 'lr': 0.0029226566353805624}. Best is trial 2 with value: 0.82109375.
[I 2022-03-26 07:25:04,657] Trial 3 finished with value

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  66
  Number of complete trials:  34
Best trial:
  Value:  0.86171875
  Params: 
    n_layers: 1
    n_units_l0: 118
    dropout_l0: 0.23229479872050213
    optimizer: Adam
    lr: 0.005031299309471552


In [18]:
fig = optuna.visualization.plot_contour(study, params=["n_units_l0", "dropout_l0"])
fig.show()

In [17]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [16]:
fig = optuna.visualization.plot_intermediate_values(study)
fig.show()

In [15]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [19]:
fig = optuna.visualization.plot_contour(study)
fig.show()